In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

from langchain_ollama import ChatOllama

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import ollama

In [2]:
# Load the document
loader = CSVLoader(file_path=r'C:\Users\test\Desktop\DM_METADATA.csv', encoding='utf-8')
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [3]:
local_embeddings = OllamaEmbeddings(model='nomic-embed-text')

vectorstore = Chroma.from_documents(documents=docs, embedding=local_embeddings)

In [ ]:
# context = """Ты — экспертный помощник по Аналитическому центру.
# Твоя задача — помогать пользователю работать с базой данных: объяснять таблицы, генерировать SQL-запросы, оптимизировать их, находить ошибки и предлагать улучшения.
# Отвечай четко, коротко и на русском языке. Если генерируешь SQL-запрос — пиши его в блоке кода. Если непонятно, что хочет пользователь — задай уточняющий вопрос.

# Описание Аналитического центра
# База данных: Vertica 24.1.0
# Схемы:
#   STAGE_DO - Временное или первичное хранилище «сырых» данных, загруженных из источников.
#   DWH - Хранилище данных
#   DM - Витрины данных
#   SANDBOX - Песочница: изолированная среда, где аналитики, дата-сайентисты и разработчики могут экспериментировать с данными, не нарушая основную архитектуру хранилища.

# Модель данных DWH - Data Vault 2. Нейминг таблиц в DWH: Начинается на h - Hub, начинается на l - link, начинается на s - satellite."""
# question = """Where is the info about Contracts?"""
# docs = vectorstore.similarity_search(question, 50)
# len(docs)

DEFINE PARAMS

In [4]:
models = ['llama3.1:latest',
          'deepseek-r1:latest',
          'mistral:latest']

In [5]:
questions = ['What is the purpose of the table DWH.S_LEGAL_ENTITY_GENERAL?',
             'How can I calculate the number of records in the table DWH.S_LEGAL_ENTITY_GENERAL?',
             'How can I retrieve the latest LOAD_DATE in the table DWH.S_LEGAL_ENTITY_GENERAL starting from the date 2024-12-31?',
             'Give the name of the client with the largest total contract amount starting from the date 2024-12-31.',
             'Where are the clients of Baiterek stored?',
             'How can I output the number of contracts broken down by DO?',
             'Which link is responsible for connecting contracts and clients?',
             'Where is the information about contracts stored?',
             'Where is the information about clients stored?',
             'Where is the project amount stored?',
             'Which data mart is responsible for CUSTOMER_360?']

In [6]:
results = []

TEST MODELS

In [7]:
for model_name in models:
    model = ChatOllama(model=model_name)
    
    for question in questions:
        prompt = ChatPromptTemplate.from_template("""You are an expert assistant for the Analytics Center.
        Your task is to help the user work with the database: explain tables, generate SQL queries, optimize them, find errors, and suggest improvements.
        Respond clearly, briefly, and in english. If you generate an SQL query — write it in a code block. If the user's request is unclear — ask a clarifying question.
        
        Analytics Center Description
        Database: Vertica 24.1.0
        Schemas:
        
        STAGE_DO – Temporary or primary storage of "raw" data loaded from sources. These are "raw" data loaded directly from sources, often without cleaning or normalization. Using it directly is risky: the data may be dirty, incomplete, or unstable.
        
        DWH – Data warehouse. This is a normalized, verified, and consistent data warehouse. This is usually the best choice: the data here has already been processed, cleaned, and standardized.
        
        DM – Data marts. These are aggregated, specialized datasets prepared for specific tasks or reports. Very convenient for targeted analytical queries, but not always suitable if detailed data is needed. Use tables from these schema first if it's possible
        
        SANDBOX – Sandbox: an isolated environment where analysts, data scientists, and developers can experiment with data without disrupting the core data warehouse architecture.
        
        Database metadata in JSON: {docs}
        
        Answer the question: """ + question
        )
        
        
        # Convert loaded documents into strings by concatenating their content
        # and ignoring metadata
        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)
        
        
        chain = {"docs": format_docs} | prompt | model | StrOutputParser()
                
        docs = vectorstore.similarity_search(question, 50)

        response = chain.invoke(docs)

        response_rus = ollama.chat(
            model=model_name,
            messages=[
                {'role': 'user', 'content': f'Translate the text provided below into Russian. However, do not change the names of tables and columns. Table names start with STAGE_DO, DWH, DM, or SANDBOX. The names of both tables and columns are written in uppercase letters. "{response}"'}
            ]
        )
        
        results.append(['nomic-embed-text', model_name, question, response, response_rus])

In [13]:
import pandas as pd

In [30]:
df = pd.DataFrame(results)

In [32]:
df.columns =['embedding_model','llm_model','question', 'response', 'response_rus']

In [37]:
df.to_csv(r'C:\Users\test\Desktop\DBAssistant\results.csv', index=False, encoding='utf-8-sig')